In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

In [ ]:
import pandas as pd  # Import Pandas

file_path = "/content/drive/MyDrive/Project/new_db.csv"  # Update this based on your Drive path
df = pd.read_csv(file_path)  # Read the CSV file
print(f"Dataset loaded with {df.shape[0]} rows and {df.shape[1]} columns.")


Dataset loaded with 42841 rows and 5 columns.


In [ ]:
!pip install transformers torchtext torch_geometric datasets nltk pandas scikit-learn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 103.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 81.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import RobertaTokenizer, RobertaModel
from torch.utils.data import Dataset, DataLoader
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import nltk
import re

# Download NLTK resources
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
!pip install contractions


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.3/118.3 kB 10.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import re
import torch
import spacy
import contractions
import nltk
from transformers import RobertaTokenizer
from nltk.corpus import stopwords

# Download required NLTK resources (if not already available)
nltk.download('stopwords')
nltk.download('punkt')

# Load English stopwords as a set for faster lookups
stop_words = set(stopwords.words('english'))

# Load dataset
file_path = "/content/drive/MyDrive/Project/new_db.csv"  # Update if needed
df = pd.read_csv(file_path)
print(f"Dataset loaded with {df.shape[0]} rows and {df.shape[1]} columns.")

# Keep necessary columns
columns_to_keep = df[["subject", "date", "label"]]

# Combine title and text
df["Statement"] = df["title"].astype(str) + " " + df["text"].astype(str)

# Drop unnecessary columns before processing
df.drop(columns=["title", "text"], inplace=True, errors="ignore")

# Drop missing values
df.dropna(inplace=True)
print(f"Dataset after dropping missing values: {df.shape[0]} rows.")

# Load spaCy model (disable unnecessary components for speed)
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])

# **Fast text cleaning using spaCy pipe (batch processing)**
def clean_texts(texts):
    texts = [contractions.fix(text.lower()) for text in texts]  # Expand contractions & lowercase
    texts = [re.sub(r'\W+', ' ', text).strip() for text in texts]  # Remove punctuation & extra spaces

    docs = nlp.pipe(texts, batch_size=50)  # Process in batches
    return [" ".join([token.lemma_ for token in doc if token.text not in stop_words]) for doc in docs]

print("Applying text cleaning...")
df["Statement"] = clean_texts(df["Statement"].tolist())
print("Text cleaning complete.")

# Load RoBERTa tokenizer
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

# **Fast batch tokenization**
print("Tokenizing text...")
df["tokenized"] = tokenizer.batch_encode_plus(
    df["Statement"].tolist(),
    truncation=True,
    padding="max_length",
    max_length=128
)["input_ids"]
print("Tokenization complete.")

# Convert labels to numeric (Fake = 0, Real = 1)
df["label"] = df["label"].astype("category").cat.codes

# Ensure correct column order
df = df[["subject", "date", "label", "Statement", "tokenized"]]

# Save preprocessed dataset
output_file = "/content/drive/MyDrive/Project/final_preprocessed_news.csv"
df.to_csv(output_file, index=False)

print(f"Preprocessing complete. File saved as '{output_file}'.")


In [ ]:
total_rows = df.shape[0]
print("Total rows in new_db.csv:", total_rows)

In [ ]:
import pandas as pd

# Load the final preprocessed dataset (Update the correct path if needed)
file_path = "/content/drive/MyDrive/Project/final_preprocessed_news.csv"
df = pd.read_csv(file_path)

# Check if the 'label' column exists
if "label" in df.columns:
    # Count the number of Real (1) and Fake (0) news articles
    label_counts = df["label"].value_counts()
    print("Label distribution:\n", label_counts)
else:
    print("Error: 'label' column not found in the dataset!")


In [ ]:
total_rows = df.shape[0]
print("Total rows in final_preprocessed_news.csv:", total_rows)

In [ ]:
import pandas as pd
import torch

# ✅ Load the processed dataset
file_path = "/content/drive/MyDrive/Project/final_preprocessed_news.csv"  # Update if needed
df_processed = pd.read_csv(file_path)

# ✅ Print the first 5 rows to check
print("✅ Dataset Loaded Successfully!")
print(df_processed.head())  # Show the first 5 rows


In [ ]:
import pandas as pd
import ast  # To safely convert string representation of list to actual list
import torch

# Load the preprocessed dataset (Update the path if running in Colab)
file_path = "/content/drive/MyDrive/Project/final_preprocessed_news.csv"  # Adjust if needed
df_processed = pd.read_csv(file_path)

# Ensure the tokenized column exists
if "tokenized" not in df_processed.columns:
    raise ValueError("The 'tokenized' column is missing in the dataset!")

# Drop rows where "tokenized" is NaN (to prevent errors)
df_processed = df_processed.dropna(subset=["tokenized"])

# Convert the tokenized column from string representation to actual lists
def safe_convert(text):
    try:
        return torch.tensor(ast.literal_eval(text)) if pd.notna(text) else torch.tensor([])
    except:
        return torch.tensor([])

df_processed["tokenized"] = df_processed["tokenized"].apply(safe_convert)

# Filter out empty tensors before stacking
valid_tensors = [t for t in df_processed["tokenized"] if t.numel() > 0]

if len(valid_tensors) > 0:
    input_tensors = torch.stack(valid_tensors)
    print("Tokenized data converted to PyTorch tensor shape:", input_tensors.shape)
else:
    print("Error: No valid tokenized tensors found!")


In [ ]:
import torch
import os

# Check if valid_tensors exist before stacking
if len(valid_tensors) > 0:
    input_tensors = torch.stack(valid_tensors)

    # Define the save path (Google Drive if in Colab, or current directory)
    save_path = "/content/drive/MyDrive/Project/tokenized_data.pt"  # Adjust as needed

    # Save the tensor data to a file
    torch.save(input_tensors, save_path)

    print(f"Tokenized data saved as '{save_path}'")
else:
    print("Error: No valid tokenized tensors to save!")


In [ ]:
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader

# Load the preprocessed dataset (Update path if needed)
file_path = "/content/drive/MyDrive/Project/final_preprocessed_news.csv"  # Update if running in Colab
df_processed = pd.read_csv(file_path)

# Ensure 'label' column is numeric (convert & handle errors)
df_processed["label"] = pd.to_numeric(df_processed["label"], errors="coerce")  # Converts invalid values to NaN

# Drop rows with NaN labels (if any)
df_processed = df_processed.dropna(subset=["label"])

# Convert labels to integer
df_processed["label"] = df_processed["label"].astype(int)

# Load tokenized tensors safely
tokenized_path = "/content/drive/MyDrive/Project/tokenized_data.pt"  # Update if needed
tokenized_tensors = torch.load(tokenized_path)  # Load PyTorch tensor file

# Ensure the dataset size matches
num_samples = min(len(df_processed), len(tokenized_tensors))
df_processed = df_processed.iloc[:num_samples]  # Trim dataset if needed
tokenized_tensors = tokenized_tensors[:num_samples]  # Trim tokenized data if needed

# Convert labels into PyTorch tensor
labels_tensor = torch.tensor(df_processed["label"].tolist(), dtype=torch.long)

# Define Dataset class
class NewsDataset(Dataset):
    def __init__(self, tokenized_data, labels):
        self.tokenized_data = tokenized_data
        self.labels = labels  # Ensure labels are also tensors

    def __len__(self):
        return len(self.tokenized_data)

    def __getitem__(self, idx):
        return self.tokenized_data[idx], self.labels[idx]

# Create dataset
news_dataset = NewsDataset(tokenized_tensors, labels_tensor)

# Create DataLoader
batch_size = 32
train_loader = DataLoader(news_dataset, batch_size=batch_size, shuffle=True)

# Debugging info
print(f"✅ DataLoader is ready with batch size {batch_size}")
print(f"Total samples in dataset: {len(news_dataset)}")
print(f"Unique labels count: {df_processed['label'].value_counts().to_dict()}")


In [ ]:
import torch
import torch.nn as nn
from transformers import RobertaModel

class RoBERTa_GNN_HAN(nn.Module):
    def __init__(self, roberta_model_name="roberta-base", hidden_dim=128, num_classes=2):
        super(RoBERTa_GNN_HAN, self).__init__()

        # Load pre-trained RoBERTa
        self.roberta = RobertaModel.from_pretrained(roberta_model_name)
        self.fc1 = nn.Linear(self.roberta.config.hidden_size, hidden_dim)

        # GNN Layer - Modeling "Statement vs Subject" Relationship
        self.gnn = nn.Linear(hidden_dim, hidden_dim)  # Replace with actual GNN implementation

        # HAN Layer - Attention over Subject Context
        self.attention = nn.Linear(hidden_dim, 1)

        # Output Layer
        self.fc2 = nn.Linear(hidden_dim, num_classes)
        self.relu = nn.ReLU()

    def forward(self, input_ids, attention_mask, subject_embedding):
        """
        input_ids: Tokenized input text (statements)
        attention_mask: Attention mask for RoBERTa
        subject_embedding: Encoded subject information (additional input for GNN)
        """

        # Step 1: RoBERTa for Statement Encoding
        roberta_output = self.roberta(input_ids=input_ids, attention_mask=attention_mask)
        hidden_states = roberta_output.last_hidden_state[:, 0, :]  # CLS token representation

        # Step 2: Feature Extraction
        x = self.relu(self.fc1(hidden_states))

        # Step 3: GNN Layer - Incorporate Subject Information
        x = self.relu(self.gnn(x + subject_embedding))  # Incorporate subject data into GNN

        # Step 4: HAN - Attention Mechanism
        attention_weights = torch.softmax(self.attention(x), dim=1)
        x = x * attention_weights  # Apply attention

        # Step 5: Classification
        x = self.fc2(x)

        return x


In [ ]:
import torch
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset

# Load the preprocessed dataset (Update path if needed)
file_path = "/content/drive/MyDrive/Project/final_preprocessed_news.csv"  # Update if needed
df_processed = pd.read_csv(file_path)

# Ensure labels are numeric and properly formatted
df_processed["label"] = pd.to_numeric(df_processed["label"], errors="coerce").fillna(0).astype(int)

# Load tokenized data safely
tokenized_path = "/content/drive/MyDrive/Project/tokenized_data.pt"  # Update if needed
try:
    tokenized_tensors = torch.load(tokenized_path)  # Load PyTorch tensor file
except FileNotFoundError:
    raise ValueError(f"Error: File '{tokenized_path}' not found!")

# Ensure dataset and tensor alignment
num_samples = min(len(df_processed), len(tokenized_tensors))
df_processed = df_processed.iloc[:num_samples]  # Trim dataset if needed
tokenized_tensors = tokenized_tensors[:num_samples]  # Trim tokenized data if needed

# Convert labels into tensor format
labels_tensor = torch.tensor(df_processed["label"].tolist(), dtype=torch.long)

# Create dataset
train_dataset = TensorDataset(tokenized_tensors, labels_tensor)

# Define DataLoader
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Debugging Info
print(f"✅ DataLoader ready with batch size {batch_size}")
print(f"Total samples in dataset: {len(train_dataset)}")
print(f"Unique labels count: {df_processed['label'].value_counts().to_dict()}")


In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import pandas as pd

# ✅ Load the processed dataset
file_path = "/content/drive/MyDrive/Project/final_preprocessed_news.csv"  # Update if needed
df_processed = pd.read_csv(file_path)

# ✅ Load tokenized data
tokenized_path = "/content/drive/MyDrive/Project/tokenized_data.pt"
tokenized_tensors = torch.load(tokenized_path)

# ✅ Convert labels to tensors
labels_tensor = torch.tensor(df_processed["label"].values, dtype=torch.long)

# ✅ Ensure dataset and labels match
num_samples = min(len(tokenized_tensors), len(labels_tensor))
tokenized_tensors = tokenized_tensors[:num_samples]
labels_tensor = labels_tensor[:num_samples]

# ✅ Split into Train & Test (80% Train, 20% Test)
train_inputs, test_inputs, train_labels, test_labels = train_test_split(
    tokenized_tensors, labels_tensor, test_size=0.2, random_state=42
)

# ✅ Create TensorDataset
train_dataset = TensorDataset(train_inputs, train_labels)
test_dataset = TensorDataset(test_inputs, test_labels)

# ✅ Define DataLoader (MUST BE CREATED)
batch_size = 16  # Change as needed
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)

print(f"✅ DataLoader ready! Train: {len(train_dataset)} samples, Test: {len(test_dataset)} samples")


In [ ]:
import torch
import torch.optim as optim
import time
import os
from torch.amp import autocast, GradScaler  # ✅ Updated for PyTorch 2.1+
from torch.nn.utils import clip_grad_norm_
from google.colab import drive  # ✅ Google Drive support

# ✅ Mount Google Drive
drive.mount('/content/drive')

# ✅ Define save paths in Google Drive
save_dir = "/content/drive/MyDrive/Project/Checkpoints"
os.makedirs(save_dir, exist_ok=True)
best_model_path = os.path.join(save_dir, "best_model.pt")  # ✅ Save the best model
training_log_path = os.path.join(save_dir, "training_log.txt")  # ✅ Save training logs

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = RoBERTa_GNN_HAN().to(device)

optimizer = optim.AdamW(model.parameters(), lr=2e-5, weight_decay=1e-4)  # ✅ L2 Regularization
loss_fn = torch.nn.CrossEntropyLoss()
scaler = GradScaler("cuda")  # ✅ Updated Syntax

# ✅ Function to save model checkpoint
def save_checkpoint(epoch, model, optimizer, loss, path):
    checkpoint = {
        "epoch": epoch,
        "model_state_dict": model.state_dict(),
        "optimizer_state_dict": optimizer.state_dict(),
        "loss": loss
    }
    torch.save(checkpoint, path)
    print(f"📌 Checkpoint saved at epoch {epoch+1} ✅")

# ✅ Ensure train_loader is defined
if 'train_loader' not in globals():
    raise ValueError("🚨 Error: `train_loader` is not defined!")

# ✅ Training loop (NO Early Stopping)
num_epochs = 10  # Train for full epochs
start_time = time.time()
training_log = []

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    epoch_start_time = time.time()

    for step, batch in enumerate(train_loader, 1):
        if batch is None:
            continue  # ✅ Skip empty batches

        if len(batch) == 2:  # Handle missing attention_mask
            input_ids, labels = batch
            input_ids, labels = input_ids.to(device), labels.to(device)
            attention_mask = torch.ones_like(input_ids, device=device)  # ✅ Ensure a valid tensor
            subject_embedding = torch.zeros_like(input_ids, device=device)  # ✅ Dummy subject embedding (replace later)
        else:
            input_ids, attention_mask, subject_embedding, labels = batch
            input_ids, attention_mask, subject_embedding, labels = (
                input_ids.to(device), attention_mask.to(device), subject_embedding.to(device), labels.to(device)
            )

        optimizer.zero_grad()

        with autocast("cuda"):  # ✅ Updated Mixed Precision Syntax
            outputs = model(input_ids, attention_mask=attention_mask, subject_embedding=subject_embedding)
            loss = loss_fn(outputs, labels)

        scaler.scale(loss).backward()
        clip_grad_norm_(model.parameters(), max_norm=1.0)  # ✅ Prevent Exploding Gradients
        scaler.step(optimizer)
        scaler.update()

        total_loss += loss.item()

    epoch_time = (time.time() - epoch_start_time) / 60  # ✅ Convert to minutes
    avg_loss = total_loss / len(train_loader)

    log_entry = f"\n🔹 Epoch {epoch+1}/{num_epochs} | Avg Loss: {avg_loss:.4f} | Time: {epoch_time:.2f} min"
    training_log.append(log_entry)
    print(log_entry)

    # ✅ Save model at every epoch (NO early stopping)
    save_checkpoint(epoch, model, optimizer, avg_loss, best_model_path)

total_time = (time.time() - start_time) / 60  # ✅ Convert to minutes
final_log = f"\n🎯 Training complete in {total_time:.2f} minutes! 🚀"
training_log.append(final_log)
print(final_log)

# ✅ Save Training Log to Google Drive
with open(training_log_path, "w") as log_file:
    log_file.writelines("\n".join(training_log))

print(f"\n📄 Training log saved at: {training_log_path}")
print(f"\n✅ Best model saved at: {best_model_path}")


In [ ]:
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader

# ✅ Split into train and test sets (80% Train, 20% Test)
train_inputs, test_inputs, train_labels, test_labels = train_test_split(
    tokenized_tensors, labels_tensor, test_size=0.2, random_state=42
)

# ✅ Create TensorDataset
train_dataset = TensorDataset(train_inputs, train_labels)
test_dataset = TensorDataset(test_inputs, test_labels)

# ✅ Define DataLoaders
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print(f"✅ DataLoader ready! Train: {len(train_dataset)} samples, Test: {len(test_dataset)} samples")


In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import pandas as pd

# ✅ Load the preprocessed dataset
file_path = "/content/drive/MyDrive/Project/final_preprocessed_news.csv"  # Update path if needed
df_processed = pd.read_csv(file_path)

# ✅ Ensure 'label' column exists
if "label" not in df_processed.columns:
    raise KeyError("🚨 Error: 'label' column not found in df_processed! Available columns: ", df_processed.columns)

# ✅ Load tokenized data
tokenized_path = "/content/drive/MyDrive/Project/tokenized_data.pt"
tokenized_tensors = torch.load(tokenized_path)

# ✅ Convert labels to tensors
labels_tensor = torch.tensor(df_processed["label"].values, dtype=torch.long)

# ✅ Print Debugging Information
print(f"🔍 Tokenized data samples: {len(tokenized_tensors)}")
print(f"🔍 Label samples: {len(labels_tensor)}")

# ✅ Fix dataset size mismatch
num_samples = min(len(tokenized_tensors), len(labels_tensor))
tokenized_tensors = tokenized_tensors[:num_samples]
labels_tensor = labels_tensor[:num_samples]

print(f"✅ Fixed dataset sizes! New size: {num_samples} samples")

# ✅ Split into train and test sets (80% train, 20% test)
train_inputs, test_inputs, train_labels, test_labels = train_test_split(
    tokenized_tensors, labels_tensor, test_size=0.2, random_state=42
)

# ✅ Create TensorDataset
train_dataset = TensorDataset(train_inputs, train_labels)
test_dataset = TensorDataset(test_inputs, test_labels)

# ✅ Define DataLoaders (Optimized for Speed)
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)

print(f"✅ DataLoader ready! Train: {len(train_dataset)} samples, Test: {len(test_dataset)} samples")


In [ ]:
import torch

# ✅ Define device (GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ✅ Load the trained model checkpoint
checkpoint_path = "/content/drive/MyDrive/Project/Checkpoints/best_model.pt"  # Update path if needed
checkpoint = torch.load(checkpoint_path, map_location=device)

# ✅ Initialize the model (Ensure the same architecture as the saved model)
model = RoBERTa_GNN_HAN().to(device)  # Replace with your actual model class

# ✅ Load the saved weights
model.load_state_dict(checkpoint["model_state_dict"])
model.eval()  # Set model to evaluation mode

print("✅ Model successfully loaded and ready for evaluation!")


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# ✅ Ensure model is in evaluation mode
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in test_loader:
        if len(batch) == 2:
            input_ids, labels = batch
            subject_embedding = torch.zeros((input_ids.shape[0], 128), device=device)  # Dummy tensor
        else:
            input_ids, labels, subject_embedding = batch

        # ✅ Move to GPU/CPU
        input_ids, labels = input_ids.to(device), labels.to(device)
        subject_embedding = subject_embedding.to(device)

        # ✅ Forward pass
        outputs = model(input_ids, attention_mask=torch.ones_like(input_ids, device=device), subject_embedding=subject_embedding)

        # ✅ Convert logits to predicted class
        predictions = torch.argmax(outputs, dim=1)

        # ✅ Collect results
        all_preds.extend(predictions.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# ✅ Compute Metrics
accuracy = accuracy_score(all_labels, all_preds) * 100
precision = precision_score(all_labels, all_preds, average='weighted') * 100
recall = recall_score(all_labels, all_preds, average='weighted') * 100
f1 = f1_score(all_labels, all_preds, average='weighted') * 100

# ✅ Print Results
print(f"✅ Model Accuracy: {accuracy:.2f}%")
print(f"✅ Precision: {precision:.2f}%")
print(f"✅ Recall: {recall:.2f}%")
print(f"✅ F1-Score: {f1:.2f}%")


In [ ]:
import torch
from transformers import RobertaTokenizer, BartForConditionalGeneration, BartTokenizer

# ✅ Load summarization model (BART)
summarizer = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")
summarizer_tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")

# ✅ Load the trained Fake News Detection model
checkpoint_path = "/content/drive/MyDrive/Project/Checkpoints/best_model.pt"  # Update if needed
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ✅ Initialize the Fake News Detection Model (Ensure same architecture as saved model)
model = RoBERTa_GNN_HAN().to(device)  # Replace with your actual model class

# ✅ Load model weights
checkpoint = torch.load(checkpoint_path, map_location=device)
model.load_state_dict(checkpoint["model_state_dict"])
model.eval()  # Set model to evaluation mode

print("✅ Model successfully loaded!")

# ✅ Load tokenizer
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")  # Update if using a different tokenizer

# ✅ Function to summarize the article
def summarize_article(article):
    inputs = summarizer_tokenizer(article, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = summarizer.generate(inputs["input_ids"], max_length=50, min_length=5, length_penalty=2.0, num_beams=4, early_stopping=True)
    headline = summarizer_tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return headline

# ✅ Function to preprocess and predict
def predict_news(model, tokenizer, text, subject):
    model.eval()  # Ensure model is in eval mode

    # ✅ Tokenize input text
    inputs = tokenizer(text, padding='max_length', truncation=True, max_length=512, return_tensors="pt")

    # ✅ Encode subject (Modify if using categorical embeddings)
    subject_embedding = torch.tensor([subject], dtype=torch.float32).to(device)  # Adjust based on model structure

    # ✅ Move inputs to device
    inputs = {key: value.to(device) for key, value in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs, subject_embedding=subject_embedding)
        prediction = torch.argmax(outputs, dim=1).item()  # 0 = Fake, 1 = Real

    return "✅ Real News" if prediction == 1 else "❌ Fake News"

# ✅ Example news article
news_text = """The government has announced a secret plan to replace all paper currency with digital-only transactions overnight. According to anonymous sources, citizens will wake up to find their cash invalid, forcing them to register for a government-controlled financial system. Experts warn this move is a step toward total economic surveillance, though officials have denied such claims."""

subject_encoded = 3  # Example encoding (Modify based on your model's subject input format)

# ✅ Generate a headline
headline = summarize_article(news_text)
print("📰 Summarized Headline:", headline)

# ✅ Predict using summarized headline
result = predict_news(model, tokenizer, headline, subject_encoded)
print("📰 Prediction:", result)

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Load pre-trained RoBERTa model for sentiment analysis
model_name = "cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Function to predict sentiment
def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
        scores = torch.softmax(outputs.logits, dim=1)
        predicted_class = torch.argmax(scores).item()

    labels = ["Negative", "Neutral", "Positive"]
    return labels[predicted_class]

# Example usage
news_text = "New Policy Proposal Debated in Parliament"
sentiment = predict_sentiment(news_text)
print(f"Sentiment: {sentiment}")

In [ ]:
import torch
from transformers import RobertaTokenizer, BartForConditionalGeneration, BartTokenizer, AutoTokenizer, AutoModelForSequenceClassification

# ✅ Load summarization model (BART)
summarizer = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")
summarizer_tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")

# ✅ Load the trained Fake News Detection model
checkpoint_path = "/content/drive/MyDrive/Project/Checkpoints/best_model.pt"  # Update if needed
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ✅ Initialize Fake News Detection Model (Ensure same architecture)
model = RoBERTa_GNN_HAN().to(device)  # Replace with your actual model class

# ✅ Load model weights
checkpoint = torch.load(checkpoint_path, map_location=device)
model.load_state_dict(checkpoint["model_state_dict"])
model.eval()  # Set model to evaluation mode

print("✅ Fake News Model Loaded Successfully!")

# ✅ Load tokenizer
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")  # Update if using a different tokenizer

# ✅ Load Sentiment Analysis model (RoBERTa Sentiment)
sentiment_model_name = "cardiffnlp/twitter-roberta-base-sentiment"
sentiment_tokenizer = AutoTokenizer.from_pretrained(sentiment_model_name)
sentiment_model = AutoModelForSequenceClassification.from_pretrained(sentiment_model_name)
sentiment_model.eval()  # Set model to evaluation mode

print("✅ Sentiment Analysis Model Loaded Successfully!")

# ✅ Function to summarize the article
def summarize_article(article):
    inputs = summarizer_tokenizer(article, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = summarizer.generate(inputs["input_ids"], max_length=50, min_length=5, length_penalty=2.0, num_beams=4, early_stopping=True)
    headline = summarizer_tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return headline

# ✅ Function to predict Fake/Real news
def predict_news(model, tokenizer, text, subject):
    model.eval()  # Ensure model is in eval mode

    # ✅ Tokenize input text
    inputs = tokenizer(text, padding='max_length', truncation=True, max_length=512, return_tensors="pt")

    # ✅ Encode subject properly (Modify if using categorical embeddings)
    subject_embedding = torch.tensor([subject], dtype=torch.float32).to(device)

    # ✅ Move inputs to device
    input_ids = inputs["input_ids"].to(device)
    attention_mask = inputs["attention_mask"].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, subject_embedding=subject_embedding)
        prediction = torch.argmax(outputs, dim=1).item()  # 0 = Fake, 1 = Real

    return "✅ Real News" if prediction == 1 else "❌ Fake News"

# ✅ Function to predict sentiment (Positive/Neutral/Negative)
def predict_sentiment(text):
    inputs = sentiment_tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = sentiment_model(**inputs)
        scores = torch.softmax(outputs.logits, dim=1)
        predicted_class = torch.argmax(scores).item()

    labels = ["Negative", "Neutral", "Positive"]
    return labels[predicted_class]

# ✅ Example news article
news_text = """The U.S. Senate has passed a bipartisan bill aimed at strengthening election
 security ahead of the upcoming presidential elections. The legislation, which received
 support from both parties, allocates additional funding for cybersecurity measures and
 voter protection. Lawmakers emphasized the need to safeguard democracy against foreign
  interference and misinformation. The bill now moves to the House of Representatives for final approval."""

subject_encoded = 3  # Example encoding (Modify based on your model's subject input format)

# ✅ Generate a headline
headline = summarize_article(news_text)
print("📰 Summarized Headline:", headline)

# ✅ Predict Fake/Real News using summarized headline
news_prediction = predict_news(model, tokenizer, headline, subject_encoded)
print("📰 Fake News Prediction:", news_prediction)

# ✅ Predict sentiment of the news
news_sentiment = predict_sentiment(headline)
print("📊 Sentiment Analysis:", news_sentiment)


In [ ]:
import torch
from transformers import RobertaTokenizer, AutoTokenizer, AutoModelForSequenceClassification

# ✅ Load the trained Fake News Detection model
checkpoint_path = "/content/drive/MyDrive/Project/Checkpoints/best_model.pt"  # Update if needed
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ✅ Initialize Fake News Detection Model (Ensure same architecture)
model = RoBERTa_GNN_HAN().to(device)  # Replace with your actual model class

# ✅ Load model weights
checkpoint = torch.load(checkpoint_path, map_location=device)
model.load_state_dict(checkpoint["model_state_dict"])
model.eval()  # Set model to evaluation mode

print("✅ Fake News Model Loaded Successfully!")

# ✅ Load tokenizer
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")  # Update if using a different tokenizer

# ✅ Function to predict Fake/Real news
def predict_news(model, tokenizer, text, subject):
    model.eval()  # Ensure model is in eval mode

    # ✅ Tokenize input text
    inputs = tokenizer(text, padding='max_length', truncation=True, max_length=512, return_tensors="pt")

    # ✅ Encode subject properly (Modify if using categorical embeddings)
    subject_embedding = torch.tensor([subject], dtype=torch.float32).to(device)

    # ✅ Move inputs to device
    input_ids = inputs["input_ids"].to(device)
    attention_mask = inputs["attention_mask"].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, subject_embedding=subject_embedding)
        prediction = torch.argmax(outputs, dim=1).item()  # 0 = Fake, 1 = Real

    return "✅ Real News" if prediction == 1 else "❌ Fake News"

# ✅ Example news article
news_text = """Politician Admits to Controlling the Weather with Hidden Technology."""

subject_encoded = 3

# ✅ Predict Fake/Real News
news_prediction = predict_news(model, tokenizer, news_text, subject_encoded)
print("📰 Fake News Prediction:", news_prediction)


In [ ]:
!pip install flask-ngrok


In [ ]:
!pip install flask transformers googletrans==4.0.0-rc1


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 3.9 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17396 sha256=454aa71e9259d47c74e2bb6ba33f3282c7b8fc5dd90306006caef06148b0bc55
  Stored in directory: /root/.cache/pip/wheels/39/17/6f/66a045ea3d168826074691b4b787b8f324d3f646d755443fda
Successfully built googletrans
  Attempting uninstall: hyperframe
    Found existing installation: hyperframe 6.1.0
    Uninstalling hyperfra

In [ ]:

!ngrok config add-authtoken 2tFeAis3sCd5sYSkXCGiAXey4QD_7MuDVoGFZb9LDTrct7MX5

/bin/bash: line 1: ngrok: command not found


In [ ]:
!cat /root/.config/ngrok/ngrok.yml


cat: /root/.config/ngrok/ngrok.yml: No such file or directory


In [ ]:
!pip install pyngrok


In [ ]:
import torch
import torch.nn as nn
from transformers import RobertaModel, BartForConditionalGeneration, BartTokenizer, AutoTokenizer, AutoModelForSequenceClassification, RobertaTokenizer
from googletrans import Translator
from flask import Flask, render_template, request
from pyngrok import ngrok  # ✅ For Colab Hosting
import os

app = Flask(__name__)

# ✅ Define RoBERTa + GNN + HAN Model
class RoBERTa_GNN_HAN(nn.Module):
    def __init__(self, roberta_model_name="roberta-base", hidden_dim=128, num_classes=2):
        super(RoBERTa_GNN_HAN, self).__init__()
        self.roberta = RobertaModel.from_pretrained(roberta_model_name)
        self.fc1 = nn.Linear(self.roberta.config.hidden_size, hidden_dim)
        self.gnn = nn.Linear(hidden_dim, hidden_dim)
        self.attention = nn.Linear(hidden_dim, 1)
        self.fc2 = nn.Linear(hidden_dim, num_classes)
        self.relu = nn.ReLU()

    def forward(self, input_ids, attention_mask, subject_embedding):
        roberta_output = self.roberta(input_ids=input_ids, attention_mask=attention_mask)
        hidden_states = roberta_output.last_hidden_state[:, 0, :]
        x = self.relu(self.fc1(hidden_states))
        x = self.relu(self.gnn(x + subject_embedding))
        attention_weights = torch.softmax(self.attention(x), dim=1)
        x = x * attention_weights
        x = self.fc2(x)
        return x

# ✅ Load Models
summarizer = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")
summarizer_tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")

checkpoint_path = "/content/drive/MyDrive/Project/Checkpoints/best_model.pt"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if not os.path.exists(checkpoint_path):
    print(f"🚨 Model file not found: {checkpoint_path}")
    model = None
else:
    try:
        model = RoBERTa_GNN_HAN().to(device)
        checkpoint = torch.load(checkpoint_path, map_location=device)
        model.load_state_dict(checkpoint["model_state_dict"])
        model.eval()
        print("✅ Fake News Model Loaded Successfully!")
    except Exception as e:
        print(f"🚨 Error loading Fake News Model: {e}")
        model = None

tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
sentiment_model_name = "cardiffnlp/twitter-roberta-base-sentiment"
sentiment_tokenizer = AutoTokenizer.from_pretrained(sentiment_model_name)
sentiment_model = AutoModelForSequenceClassification.from_pretrained(sentiment_model_name)
sentiment_model.eval()
translator = Translator()

def translate_to_english(text, src_language):
    try:
        return translator.translate(text, src=src_language, dest="en").text
    except Exception as e:
        print(f"Translation Error: {e}")
        return text

def summarize_article(article):
    inputs = summarizer_tokenizer(article, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = summarizer.generate(inputs["input_ids"], max_length=50, min_length=5, length_penalty=2.0, num_beams=4, early_stopping=True)
    return summarizer_tokenizer.decode(summary_ids[0], skip_special_tokens=True)

def predict_news(model, tokenizer, text, subject=3):
    if model is None:
        return "❌ Fake News (Model Not Loaded)"
    model.eval()
    inputs = tokenizer(text, padding='max_length', truncation=True, max_length=512, return_tensors="pt")
    subject_embedding = torch.tensor([[subject]], dtype=torch.float32).to(device)
    input_ids = inputs["input_ids"].to(device)
    attention_mask = inputs["attention_mask"].to(device)
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, subject_embedding=subject_embedding)
        prediction = torch.argmax(outputs, dim=1).item()
    return "✅ Real News" if prediction == 1 else "❌ Fake News"

def predict_sentiment(text):
    inputs = sentiment_tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = sentiment_model(**inputs)
        scores = torch.softmax(outputs.logits, dim=1)
        predicted_class = torch.argmax(scores).item()
    labels = ["Negative", "Neutral", "Positive"]
    return labels[predicted_class]

@app.route("/", methods=["GET"])
def home():
    return render_template("home.html")

@app.route("/templates", methods=["GET", "POST"])
def index():
    if request.method == "POST":
        user_text = request.form.get("news_text", "").strip()
        selected_language = request.form.get("language", "en")

        if not user_text:
            return render_template("index.html", error="❌ Please enter news text.", selected_language=selected_language)

        translated_text = translate_to_english(user_text, selected_language) if selected_language != "en" else user_text
        sentiment = predict_sentiment(translated_text)
        summarized_text = summarize_article(translated_text)
        fake_news_prediction = predict_news(model, tokenizer, summarized_text)

        google_search_url = ""
        if fake_news_prediction == "✅ Real News":
            query = summarized_text.replace(" ", "+")
            google_search_url = f"https://www.google.com/search?q={query}"

        return render_template(
            "index.html",
            user_text=user_text,
            selected_language=selected_language,
            translated_text=translated_text,
            summarized_text=summarized_text,
            fake_news_prediction=fake_news_prediction,
            sentiment=sentiment,
            google_search_url=google_search_url
        )

    return render_template("index.html")

if __name__ == "__main__":
    public_url = ngrok.connect(5000)
    print(f"🔗 Public URL: {public_url}")
    app.run(port=5000)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ Fake News Model Loaded Successfully!


ERROR:pyngrok.process.ngrok:t=2025-04-24T07:24:57+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Your account is limited to 1 simultaneous ngrok agent sessions.\nYou can run multiple simultaneous tunnels from a single agent session by defining the tunnels in your agent configuration file and starting them with the command `ngrok start --all`.\nRead more about the agent configuration file: https://ngrok.com/docs/secure-tunnels/ngrok-agent/reference/config\nYou can view your current agent sessions in the dashboard:\nhttps://dashboard.ngrok.com/agents\r\n\r\nERR_NGROK_108\r\n"
ERROR:pyngrok.process.ngrok:t=2025-04-24T07:24:57+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Your account is limited to 1 simultaneous ngrok agent sessions.\nYou can run multiple simultaneous tunnels from a single agent session by defining the tunnels in your agent configuration file and starting them with the command `ngrok st

PyngrokNgrokError: The ngrok process errored on start: authentication failed: Your account is limited to 1 simultaneous ngrok agent sessions.\nYou can run multiple simultaneous tunnels from a single agent session by defining the tunnels in your agent configuration file and starting them with the command `ngrok start --all`.\nRead more about the agent configuration file: https://ngrok.com/docs/secure-tunnels/ngrok-agent/reference/config\nYou can view your current agent sessions in the dashboard:\nhttps://dashboard.ngrok.com/agents\r\n\r\nERR_NGROK_108\r\n.

In [ ]:
import torch
import torch.nn as nn
from transformers import RobertaModel, BartForConditionalGeneration, BartTokenizer, AutoTokenizer, AutoModelForSequenceClassification, RobertaTokenizer
from googletrans import Translator
from flask import Flask, render_template, request
from pyngrok import ngrok  # ✅ For Colab Hosting
import os

# ✅ Initialize Flask app
app = Flask(__name__)

# ✅ Define RoBERTa + GNN + HAN Model
class RoBERTa_GNN_HAN(nn.Module):
    def __init__(self, roberta_model_name="roberta-base", hidden_dim=128, num_classes=2):
        super(RoBERTa_GNN_HAN, self).__init__()

        # ✅ Load pre-trained RoBERTa
        self.roberta = RobertaModel.from_pretrained(roberta_model_name)
        self.fc1 = nn.Linear(self.roberta.config.hidden_size, hidden_dim)

        # ✅ GNN Layer - Modeling "Statement vs Subject" Relationship
        self.gnn = nn.Linear(hidden_dim, hidden_dim)

        # ✅ HAN Layer - Attention over Subject Context
        self.attention = nn.Linear(hidden_dim, 1)

        # ✅ Output Layer
        self.fc2 = nn.Linear(hidden_dim, num_classes)
        self.relu = nn.ReLU()

    def forward(self, input_ids, attention_mask, subject_embedding):
        roberta_output = self.roberta(input_ids=input_ids, attention_mask=attention_mask)
        hidden_states = roberta_output.last_hidden_state[:, 0, :]  # CLS token representation

        # Feature Extraction
        x = self.relu(self.fc1(hidden_states))

        # GNN Layer - Incorporate Subject Information
        x = self.relu(self.gnn(x + subject_embedding))

        # HAN - Attention Mechanism
        attention_weights = torch.softmax(self.attention(x), dim=1)
        x = x * attention_weights

        # Classification
        x = self.fc2(x)

        return x

# ✅ Load summarization model (BART)
summarizer = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")
summarizer_tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")

# ✅ Load Fake News Detection model
checkpoint_path = "/content/drive/MyDrive/Project/Checkpoints/best_model.pt"  # Update path if needed
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ✅ Check if Model File Exists
if not os.path.exists(checkpoint_path):
    print(f"🚨 Model file not found: {checkpoint_path}")
    model = None
else:
    try:
        model = RoBERTa_GNN_HAN().to(device)
        checkpoint = torch.load(checkpoint_path, map_location=device)
        model.load_state_dict(checkpoint["model_state_dict"])
        model.eval()
        print("✅ Fake News Model Loaded Successfully!")
    except Exception as e:
        print(f"🚨 Error loading Fake News Model: {e}")
        model = None

# ✅ Load tokenizer for Fake News Model
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

# ✅ Load sentiment analysis model (RoBERTa)
sentiment_model_name = "cardiffnlp/twitter-roberta-base-sentiment"
sentiment_tokenizer = AutoTokenizer.from_pretrained(sentiment_model_name)
sentiment_model = AutoModelForSequenceClassification.from_pretrained(sentiment_model_name)
sentiment_model.eval()

# ✅ Load Google Translator
translator = Translator()

# ✅ Function to translate text to English
def translate_to_english(text, src_language):
    try:
        return translator.translate(text, src=src_language, dest="en").text
    except Exception as e:
        print(f"Translation Error: {e}")
        return text  # If translation fails, return original text

# ✅ Function to summarize the article
def summarize_article(article):
    inputs = summarizer_tokenizer(article, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = summarizer.generate(inputs["input_ids"], max_length=50, min_length=5, length_penalty=2.0, num_beams=4, early_stopping=True)
    return summarizer_tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# ✅ Function to predict Fake/Real news
def predict_news(model, tokenizer, text, subject=3):
    if model is None:
        return "❌ Fake News (Model Not Loaded)"

    model.eval()
    inputs = tokenizer(text, padding='max_length', truncation=True, max_length=512, return_tensors="pt")
    subject_embedding = torch.tensor([[subject]], dtype=torch.float32).to(device)

    input_ids = inputs["input_ids"].to(device)
    attention_mask = inputs["attention_mask"].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, subject_embedding=subject_embedding)
        prediction = torch.argmax(outputs, dim=1).item()  # 0 = Fake, 1 = Real

    return "✅ Real News" if prediction == 1 else "❌ Fake News"

# ✅ Function to predict sentiment (Positive/Neutral/Negative)
def predict_sentiment(text):
    inputs = sentiment_tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = sentiment_model(**inputs)
        scores = torch.softmax(outputs.logits, dim=1)
        predicted_class = torch.argmax(scores).item()

    labels = ["Negative", "Neutral", "Positive"]
    return labels[predicted_class]

# ✅ Flask Routes
@app.route("/", methods=["GET"])  # Only allow GET for home page
def home():
    return render_template("home.html")

@app.route("/templates", methods=["GET", "POST"])  # Allow POST for analysis
def index():
    if request.method == "POST":
        user_text = request.form.get("news_text", "").strip()
        selected_language = request.form.get("language", "en")

        if not user_text:
            return render_template("index.html", error="❌ Please enter news text.", selected_language=selected_language)

        # ✅ Sentiment Analysis Logic
        if selected_language == "en":
            sentiment = predict_sentiment(user_text)  # Analyze sentiment on original English text
            translated_text = user_text  # No translation needed
        else:
            translated_text = translate_to_english(user_text, selected_language)
            sentiment = predict_sentiment(translated_text)  # Analyze sentiment on translated text

        # ✅ Summarization & Fake News Detection
        summarized_text = summarize_article(translated_text)
        fake_news_prediction = predict_news(model, tokenizer, summarized_text)

        return render_template(
            "index.html",
            user_text=user_text,
            selected_language=selected_language,
            translated_text=translated_text,
            summarized_text=summarized_text,
            fake_news_prediction=fake_news_prediction,
            sentiment=sentiment
        )

    return render_template("index.html")  # Default page when accessed via GET
# ✅ Run Flask App with Public URL in Google Colab
if __name__ == "__main__":
    public_url = ngrok.connect(5000)
    print(f"🔗 Public URL: {public_url}")
    app.run(port=5000)


KeyboardInterrupt: 

In [ ]:
!ngrok config check


In [ ]:
!cp -r "/content/drive/MyDrive/Project/templates" "/content/templates"


In [ ]:
!ls /content/templates


home.html  index.html


In [ ]:
!pip install python-telegram-bot


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 673.5/673.5 kB 26.3 MB/s eta 0:00:00


In [ ]:
pip install python-telegram-bot requests


In [ ]:
!pip install python-telegram-bot --upgrade


In [ ]:
import telegram
import requests
from telegram import Update
from telegram.ext import Application, CommandHandler, MessageHandler, filters

# Your Telegram Bot Token
TOKEN = '7882086241:AAEzpPtVVf8ELodur-_QvfFvO5mocgROfD8'
bot = telegram.Bot(token=TOKEN)

# Flask API URL (corrected URL format)
FLASK_URL = "https://f7f6-34-138-243-104.ngrok-free.app/api/analyze"  # Corrected URL

# Function to handle the command and process news
async def process_news(update: Update, context):
    # Retrieve the message text (news article)
    user_text = update.message.text

    # Get user's preferred language (or default to 'en')
    selected_language = "en"  # Example, modify based on your bot's language feature

    # Prepare the data to send to Flask API
    data = {
        "news_text": user_text,
        "language": selected_language
    }

    # Send POST request to Flask API
    try:
        response = requests.post(FLASK_URL, json=data)
        response.raise_for_status()  # Will raise an HTTPError if the response code was 4xx/5xx

        # Parse the JSON response from Flask
        result = response.json()

        if "error" in result:
            await update.message.reply_text(result["error"])
        else:
            # Display results (summarized text, fake news prediction, sentiment, etc.)
            await update.message.reply_text(f"Sentiment: {result['sentiment']}\nFake News Prediction: {result['fake_news_prediction']}\nSummary: {result['summarized_text']}")

    except requests.exceptions.RequestException as e:
        await update.message.reply_text(f"Error communicating with the Flask server: {str(e)}")

# Add your bot's handler and run it
async def start(update: Update, context):
    await update.message.reply_text('Send me some news text, and I will analyze it!')

# Use the Application class instead of Updater
application = Application.builder().token(TOKEN).build()

# Add handlers
application.add_handler(CommandHandler('start', start))
application.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, process_news))

# Start polling for updates
application.run_polling()


RuntimeError: Cannot close a running event loop

In [ ]:
!pip install nest_asyncio


In [ ]:
!pip install python-telegram-bot
!pip install flask
!pip install pyngrok
